In [41]:
import numpy as np
import quantecon as qe
from numba import njit

The state process is

$$ y_{t+1} = \rho y_t + b + \xi_{t+1} $$

where $-1 < \rho < 1$ and $\{ \xi_t \}$ is iid and uniform on $[-\sigma, \sigma]$

In [ ]:
ρ = 0.5
b = 10.0
σ = 0.5

The discount rate:

In [72]:
β = 0.9

Entry costs:

In [73]:
def ϕ(n):
    return n**2

Profit function, with parameters:

In [82]:
d = 5.0
α = 2.0

def π(n, y):
    return d + y - n**α

The stationary distribution for $y_t$ is supported on the interval

$$ I := \left[ \frac{b - \sigma}{1 - \rho}, \, \frac{b + \sigma}{1 - \rho} \right] $$

Grid over $y$ values, covering the stationary distribution:

In [83]:
y_min = (b - σ) / (1 - ρ)
y_max = (b + σ) / (1 - ρ)
y_grid_size = 5
y_grid = np.linspace(y_min, y_max, y_grid_size)

Set `n_max` to the first $n$ such that profits are negative for all possible values of $y_t$.

In [84]:
n_max = np.int(np.ceil((d + y_max**(1/α))))

In [85]:
n_max

10

In [71]:
n_size = n_max + 1

In [53]:
v_E = np.zeros((n_size, y_size))

In [ ]:
def last_firm_bellman(f_in):
    f_out = np.empty_like(f_in)
    for y in y_vec:
        t = ρ * sum([(π(n_max, yp) + f_in[]])